# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2011/07/01	Perkembangan Pengetahuan Anak Usia Dini Melalui Permainan Komputer Edukatif (Studi Kasus TK Aisyiyah 3 Salatiga)	Jurnal KomuniTi	Pelaksanaan penelitian ini bertujuan untuk mengetahui pengaruh pemanfaatan teknologi komputer terhadap peningkatan kecerdasan dan kreatifitas anak didik TK dengan berbagai jenis permainan komputer edukatif.	"Nugroho, Y.S. (2011). ""Perkembangan Pengetahuan Anak Usia Dini Melalui Permainan Komputer Edukatif (Studi Kasus TK Aisyiyah 3 Salatiga)"". KomuniTi. 3(1)."	perkembangan-anak-usia-dini	http://journals.ums.ac.id/index.php/komuniti/article/view/2966/1901
2013/09/02	"Analisa Kecepatan Proxy Squid, Safesquid dan Polipo Pada Ubuntu Server"	Jurnal Emitor	"Penelitian ini untuk membandingkan beberapa sistem operasi pada proxy server yaitu squid, safesquid dan polipo untuk mendapatkan sistem operasi proxy server yang cocok pada jaringan yang dirancang."	"Al Irsyadi, F.Y., Nugroho, Y.S., Wuryanto, 

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0, encoding='latin-1')
publications

,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2011/07/01,Perkembangan Pengetahuan Anak Usia Dini Melalu...,Jurnal KomuniTi,Pelaksanaan penelitian ini bertujuan untuk men...,"Nugroho, Y.S. (2011). ""Perkembangan Pengetahua...",perkembangan-anak-usia-dini,http://journals.ums.ac.id/index.php/komuniti/a...
1,2013/09/02,"Analisa Kecepatan Proxy Squid, Safesquid dan P...",Jurnal Emitor,Penelitian ini untuk membandingkan beberapa si...,"Al Irsyadi, F.Y., Nugroho, Y.S., Wuryanto, D. ...",analisis-squid-safesquid,https://publikasiilmiah.ums.ac.id/xmlui/handle...
2,2013/09/01,Seleksi Sekolah Menengah Lanjutan Menggunakan ...,Jurnal KomuniTi,Penelitian ini dilakukan dalam rangka mengemba...,"Nugroho, Y.S., Ulinuha, A., Aji, N.N.S. (2013)...",seleksi-sml-dengan-ahp,https://publikasiilmiah.ums.ac.id/handle/11617...
3,2014/03/01,Klasifikasi Masa Studi Mahasiswa Fakultas Komu...,Jurnal KomuniTi,Penelitian ini dilakukan untuk memanfaatkan da...,"Nugroho, Y.S., Setyawan. (2014). ""Klasifikasi ...",klasifikasi-mhs-fki-c45,http://journals.ums.ac.id/index.php/komuniti/a...
4,2015/12/01,Aplikasi Pemrediksi Masa Studi dan Predikat Ke...,Jurnal Khazanah Informatika,Penelitian ini dilakukan untuk membuat suatu a...,"Nurrohmat, M.A., Nugroho, Y.S. (2015). ""Aplika...",aplikasi-pemrediksi-kelulusan-mhs,http://journals.ums.ac.id/index.php/khif/artic...
5,2015/12/01,Klasifikasi dan Klastering Penjurusan Siswa SM...,Jurnal Khazanah Informatika,"Penerapan algoritma C4.5, Naive Bayes dan K-Me...","Nugroho, Y.S., Haryati, S.N. (2015). ""Klasifik...",klasifikasi-penentuan-jurusan-sekolah,http://journals.ums.ac.id/index.php/khif/artic...
6,2017/06/01,Sistem Klasifikasi Variabel Tingkat Penerimaan...,Jurnal Teknik Elektro,Pengembangan sistem pengklasifikasi tingkat pe...,"Nugroho, Y.S., Emiliyawati, N. (2017). ""Sistem...",aplikasi-penerimaan-mobil-RF,https://journal.unnes.ac.id/nju/index.php/jte/...
7,2016/12/01,Klasifikasi Kanker Payudara Menggunakan Algori...,Jurnal Teknik Elektro,Penelitian ini untuk mengklasifikasi jenis kan...,"Farahdiba, B.A., Nugroho, Y.S. (2016). ""Klasif...",klasifikasi-kanker-payudara-gain-ratio,https://journal.unnes.ac.id/nju/index.php/jte/...
8,2016/12/01,Prediksi Rating Film Menggunakan Metode Naive ...,Jurnal Teknik Elektro,Penelitian ini untuk mengklasifikasi dan melak...,"Pratiwi, R.W., Nugroho, Y.S. (2016). ""Prediksi...",prediksi-rating-film,https://journal.unnes.ac.id/nju/index.php/jte/...
9,2016/12/01,Implementasi Data Warehouse Dan Data Mining Un...,Jurnal Khazanah Informatika,Pengembangan sistem rekomendasi pemilihan seko...,"Nugroho, Y.S., Salma, T.D., Rokhanudin, S. (20...",dwdm-sistem-rekomendasi-sma,http://journals.ums.ac.id/index.php/khif/artic...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

01-aplikasi-pemrediksi-kelulusan-mhs.md
01-aplikasi-penerimaan-mobil-RF.md
01-dwdm-sistem-rekomendasi-sma.md
01-klasifikasi-kanker-payudara-gain-ratio.md
01-klasifikasi-mhs-fki-c45.md
01-klasifikasi-penentuan-jurusan-sekolah.md
01-perkembangan-anak-usia-dini.md
01-prediksi-rating-film.md
01-seleksi-sml-dengan-ahp.md
02-analisis-squid-safesquid.md
02-prediksi-rating-buku.md
03-identify-algorithms-incode-comment.md
06-how-different-are-different-diff.md
08-citations-in-code-comments.md
09-tda-on-social-practice.md


In [7]:
!cat ../_publications/09-tda-on-social-practice.md

---
title: "A Topological Analysis of Communication Channels for Knowledge Sharing in Contemporary GitHub Projects"
collection: publications
permalink: /publication/2019/05/09-tda-on-social-practice
excerpt: 'In this research, our goal is to analyze how communication channels share knowledge over projects. First, using the SECI model, we are able to map how knowledge is transferred through the communication channels. Then in a large-scale topology analysis of seven library package platforms, we extracted insights of how knowledge is shared by different library ecosystems within GitHub.'
date: 2019/05/09
venue: 'The Journal of Systems and Software (Minor revision)'
paperurl: 'https://arxiv.org/abs/1905.03593'
citation: 'Tantisuwankul, J., Nugroho, Y.S., Kula, R.G., Hata, H., Rungsawang, A., Leelaprute, P., &amp; Matsumoto, K. (2019). A Topological Analysis of Tacit and Explicit Knowledge Sharing on Contemporary Projects in GitHub. Submitted to JSS.'
---
In this research, our goal is to 